In [5]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import numpy as np

In [8]:
model = pickle.load(open("model.pkl", "rb"))
label_encoder = pickle.load(open("label_encoder.pkl", "rb"))
app = FastAPI()

In [7]:
def safe_join(x):
    if isinstance(x, list):
        return " ".join([str(i) for i in x])
    return str(x) if x else ""

def extract_routing_reasoning(rr):
    if not rr:
        return ""
    parts = []
    parts.append(rr.get("primaryReason", ""))
    parts.append(safe_join(rr.get("factors", [])))
    parts.append(safe_join(rr.get("suggestedActions", [])))
    return " ".join(parts)

def extract_context(ctx):
    if not ctx:
        return ""
    parts = []
    for k in ["businessImpact", "resource", "impactLevel", "timeToSLA"]:
        if k in ctx:
            parts.append(str(ctx[k]))
    return " ".join(parts)

def extract_timeline(tl):
    if not tl:
        return ""
    events = []
    for ev in tl:
        if isinstance(ev, dict) and "event" in ev:
            events.append(ev["event"])
    return " ".join(events)

In [9]:
class Incident(BaseModel):
    title: str | None = ""
    description: str | None = ""
    routingReasoning: dict | None = None
    context: dict | None = None
    timeline: list | None = None
    tags: list | None = None

In [10]:
@app.post("/predict")
def predict(incident: Incident):
    # Flatten the text (same as training)
    combined_text = (
        (incident.title or "") + " " +
        (incident.description or "") + " " +
        extract_routing_reasoning(incident.routingReasoning) + " " +
        extract_context(incident.context) + " " +
        extract_timeline(incident.timeline) + " " +
        safe_join(incident.tags)
    ).strip()

    # Prediction
    probs = model.predict_proba([combined_text])[0]
    best_idx = np.argmax(probs)
    predicted_team = label_encoder.inverse_transform([best_idx])[0]
    confidence = float(probs[best_idx])

    return {
        "team": predicted_team,
        "confidence": confidence
    }